In [8]:
import os
import cv2
import dlib
import numpy as np
import math

def txt_gen(img_name,detector,predictor):
    img = cv2.imread(img_name)
    gray=cv2.cvtColor(src=img,code=cv2.COLOR_BGR2GRAY)

    faces=detector(gray)

    pointsArray = [];

    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        landmarks=predictor(image=gray,box=face)

        for n in range(0,68):

            x=landmarks.part(n).x
            y=landmarks.part(n).y

            cv2.circle(img=img, center=(x,y),radius=5,color=(0,255,0),thickness=-1)

            points = (int(x), int(y));   

            pointsArray.append(points)
    file_name=img_name+'.txt'
    np.savetxt(file_name,pointsArray,fmt="%d")

def readImages(filetitle):
    imagesArray = []
    for file in os.listdir():
        if file==filetitle:
            img = cv2.imread(filetitle)
            img = np.float32(img)/255.0
            imagesArray.append(img)
    return imagesArray

def readPoints(filetitle):
    pointsArray=[]
    for files in os.listdir():
        if files[0:3]==filetitle[0:3] and files.endswith('.txt'):
#             filetitle=files+'.txt'
            points = [];   
            print(files)
            with open(files) as file :
                for line in file :
                    x, y = line.split()
                    points.append((int(x), int(y)))
            pointsArray.append(points)
    return pointsArray

def similarityTransform(inPoints, outPoints) :
    s60 = math.sin(60*math.pi/180);
    c60 = math.cos(60*math.pi/180);  
  
    inPts = np.copy(inPoints).tolist();
    outPts = np.copy(outPoints).tolist();
    
    xin = c60*(inPts[0][0] - inPts[1][0]) - s60*(inPts[0][1] - inPts[1][1]) + inPts[1][0];
    yin = s60*(inPts[0][0] - inPts[1][0]) + c60*(inPts[0][1] - inPts[1][1]) + inPts[1][1];
    
    inPts.append([np.int(xin), np.int(yin)]);
    
    xout = c60*(outPts[0][0] - outPts[1][0]) - s60*(outPts[0][1] - outPts[1][1]) + outPts[1][0];
    yout = s60*(outPts[0][0] - outPts[1][0]) + c60*(outPts[0][1] - outPts[1][1]) + outPts[1][1];
    
    outPts.append([np.int(xout), np.int(yout)]);
    
    tform = cv2.estimateRigidTransform(np.array([inPts]), np.array([outPts]), False);
    
    return tform;

def draw_point(img, p, color ) :
    cv2.circle(img, p, 5, color, -1, cv2.LINE_AA, 0 )    
    
    
def single_image_gen(w,h,eyecornerDst,filetitle,detector,predictor,loc_1,loc_2):
    
    allPoints=readPoints(filetitle)
    images=readImages(filetitle)
    pointsArray = [];

    imagesNorm = [];
    pointsNorm = [];
    boundaryPts = np.array([(0,0), (w/2,0), (w-1,0), (w-1,h/2), ( w-1, h-1 ), ( w/2, h-1 ), (0, h-1), (0,h/2) ]);
    pointsAvg = np.array([(0,0)]* ( len(allPoints[0]) + len(boundaryPts) ), np.float32());
    n = len(allPoints[0]);
    numImages = len(images)

    for q in range(0, numImages):
        points1 = allPoints[q];

        # Corners of the eye in input image
        eyecornerSrc  = [allPoints[q][loc_1], allPoints[q][loc_2]] ;

        # Compute similarity transform
        tform = similarityTransform(eyecornerSrc, eyecornerDst);

        # Apply similarity transformation
        img = cv2.warpAffine(images[q], tform, (w,h));

        # Apply similarity transform on points
        points2 = np.reshape(np.array(points1), (68,1,2));        

        points = cv2.transform(points2, tform);

        points = np.float32(np.reshape(points, (68, 2)));

        # Append boundary points. Will be used in Delaunay Triangulation
        points = np.append(points, boundaryPts, axis=0)

        # Calculate location of average landmark points.
        pointsAvg = pointsAvg + points / numImages;

        for p in points :
            draw_point(img, (int(p[0]),int(p[1])), (0,0,255))
            points = (int(p[0]),int(p[1]));   

            pointsArray.append(points)
        
        cv2.imwrite('dotted_'+filetitle,img*255)
        file_name=filetitle+'_dotted.txt'
        np.savetxt(file_name,pointsArray,fmt="%d")

#         cv2.imwrite('line_'+filetitle,img*255)
#         cv2.imshow(str(q+1),img)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

In [3]:
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
for file in os.listdir():
    if file[0:3]=='mwt':
        txt_gen(file,detector,predictor)

In [12]:
w=1500
h=1500

eyecornerDst = [(100, 450), (1400, 450)];

for file in os.listdir():
    if file.endswith('.jpg'):
        print(file)
        single_image_gen(w,h,eyecornerDst,file,detector,predictor,0,16)

mwt.jpg
mwt.jpg.txt


In [13]:
w=1500
h=1500

eyecornerDst = [(100, 450), (1400, 450)];

for file in os.listdir():
    if file=='initial.png':
        print(file)
        single_image_gen(w,h,eyecornerDst,file,detector,predictor,0,16)

initial.png
initial.png.txt
